## Introducción a la Visión Computacional
## Tarea 2
-----------------------
<div align="right">
Fecha de Entrega: Viernes 29, Abril 2022.
</div>

1. Diseñar un flujo de procesamiento a nivel abstracto: adquisición de imágenes, pre-procesamiento, procesamiento de imágenes, cálculo de la medición, almacenamiento.

2. Con la base de datos seleccionada, implementar un sistema de reconocimiento y/o procesamiento en el cual se realice una medida o inferencia sobre imagen. Puede ser la detección de un objeto, medición de distancias entre píxeles de alguna característica, medición del tamaño de algún objeto (en cantidad de píxeles) clasificación de la imagen, entre otros.

3. Para procesar el conjunto completo de imágenes, medir los tiempos de ejecución del proceso completo. Realizar la medición para el 50% y el 100% de la imágenes varias veces, de manera de obtener algunas figuras estadísticas: tiempo máximo, tiempo mínimo, tiempo medio, desviación estándar del tiempo.

4. Hacer resumen de los resultados del procesamiento. Para ello debe decidir una métrica para indicar si el procesamiento es satisfactorio. Para esto último algunas veces es necesario hacer la evaluación en forma manual, por lo tanto, si fuera este el caso, seleccionar un sub-conjunto de las imágenes (por ejemplo, unas 50) y comparar el desempeño del algoritmo en estas imágenes.

5. Escribir un informe o entregar el notebook con comentarios y documentación, incluyendo el análisis (comentando) los resultados.

Nota: de percatarse que el set de imágenes propuesto en la Tarea 1 no es el adecuado, se puede realizar un cambio en el dataset. Justificar el cambio. Este tipo de decisiones puede ocurrir en un proyecto.

# Base de datos de patentes de vehiculos de Chile

Base de datos recopilada por mi, en imagenes de Google, las cuales fueran fotografias de patentes de vehiculos chilenos, con esto me gustaria realizar un posterior analisis, obteniendo las letras y numeros para consultar en [autoseguro.gob.cl](https://www.autoseguro.gob.cl) de carabineros y policia de investigaciones, con el objetivo de verificar si un vehiculo tiene encargo por robo.

En este caso para el pre-procesamiento de imagenes, se aplica el filtro sobel, para poder obtener la deteccion de bordes, con esto podemos tener la capacidad de detectar medianamente bien las letras y numeros de las patentes. Obteniendo estos datos, podria verificar si estos estan con encargo por robo y asi poder informar que el vehiculo fue visto por ultima vez en una locacion especifica

## Definicion funciones para pipeline de procesamiento, conversion y reconocimiento de imagenes
Define funciones intermedias para ser referenciadas desde funcion de ejecucion de pipeline
- obtencion de imagenes desde classpath de proyecto ```get_image_list_from_path(path, extension)```
- pre-procesamiento de imagenes obtenidas en punto anterior con ```save_resized_images_resx_resy```
- procesamiento de imagenes con filtro sobel ```sobel_filter```
- reconocimiento de figuras en imagenes ```detect_vehicle_license_plate```

In [342]:
import os

import numpy as np
from PIL import Image
from numpy import *
from scipy.ndimage import sobel
from matplotlib import pyplot as plt
import cv2
import time
from pathlib import Path
import shutil

# Functiones utilitarias

In [343]:
def stopwatch(sec, function_name):
  mins = sec // 60
  sec = sec % 60
  hours = mins // 60
  mins = mins % 60
  print("Time Lapsed = {0}:{1}:{2} in function '{3}'".format(int(hours), int(mins), sec, function_name))

In [344]:
def delete_dir(path_directory):
    dir_path = Path(path_directory)
    if dir_path.exists() and dir_path.is_dir():
        shutil.rmtree(dir_path)

# Funciones de pipeline

In [345]:
'''
Obtiene las imagenes definidas en el classpath del proyecto, con los formatos que se establecen en el segundo parametro
    - path: direccion donde estaran alojadas las imagenes a procesar
    - extension: lista de extensiones que seran tomadas en cuenta para obtencion de imagenes
    - limit: limite de imagenes que obtendra desde el parametro path
'''
def get_image_list_from_path(path, extension, limit = -1):
    l = []
    for ex in extension:
        for f in os.listdir(path):
            if f.endswith(ex):
                l.append(os.path.join(path,f))

    if limit != -1:
        return l[:limit]

    return l

In [346]:
#img_from_path = get_image_list_from_path("patentes", [".png", ".jpeg"])

In [347]:
'''
Guarda las imagenes que ya fueron guardadas en el classpath, con una resolucion diferente
    - x: resolucion al eje x
    - y: resolucion al eje y
    - img_list: lista de imagenes que seran procesadas a la resolucion pasada en los parametros x e y
'''
def save_resized_images_resx_resy(x, y, img_list):
    os.makedirs("out{}x{}/patentes".format(x, y), exist_ok=True)

    for img in img_list:
        Image.open(img).convert('RGB').resize((x, y))\
            .save("out{}x{}/{}".format(x, y, img))

    return get_image_list_from_path("out{}x{}/patentes".format(x, y), [".png", ".jpeg"])

In [348]:
#resized_images_512_512 = save_resized_images_resx_resy(512, 512, img_from_path)

In [349]:
'''
De las imagenes que sean pasadas por el parametro 'img_list' (lista de paths), seran convertidas y guardadas con el filtro sobel (reconocimiento de bordes)
    - img_list: lista de imagenes que seran convertidas y guardadas con el filtro sobel
'''
def sobel_filter(img_list):

    os.makedirs("out_sobel/patentes", exist_ok=True)

    for img in img_list:
        im = array(Image.open(img).convert('L'), 'f')
        imx = zeros(im.shape)
        sobel(im, 1, imx)

        imy = zeros(im.shape)
        sobel(im, 0, imy)

        magnitude = sqrt(imx**2+imy**2)
        plt.imsave("out_sobel/{}".format("/".join(img.split("/")[-2:])), arr = magnitude, cmap = plt.get_cmap('gray'))

    return get_image_list_from_path("out_sobel/patentes", [".png", ".jpeg"])

In [350]:
#sobel_filter_images = sobel_filter(resized_images_512_512)

In [351]:
'''
Funcion orientada a reconocimiento de figuras en imagenes pasadas por el parametro 'img_list' (lista de paths)
Las imagenes quedaran con bordes en rojo
    - img_list: lista de imagenes que seran convertidas y guardadas con reconocimiento de figuras
'''
def detect_vehicle_license_plate(img_list):

    os.makedirs("out_detection/patentes", exist_ok=True)
    for img_path in img_list:

        img = cv2.imread(img_path)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        _, threshold = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
        contours, _ = cv2.findContours(
            threshold, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        i = 0
        for contour in contours:
            if i == 0:
                i = 1
                continue
            approx = cv2.approxPolyDP(
                contour, 0.01 * cv2.arcLength(contour, True), True)
            cv2.drawContours(img, [contour], 0, (0, 0, 255), 5)
            M = cv2.moments(contour)
            if M['m00'] != 0.0:
                x = int(M['m10']/M['m00'])
                y = int(M['m01']/M['m00'])

        cv2.imwrite('out_detection/{}'.format("/".join(img_path.split("/")[-2:])), img)

In [352]:
#detect_vehicle_license_plate(sobel_filter_images)

In [374]:
'''
Funcion de ejecucion de proceso de recoleccion, pre-procesamiento, procesamiento, deteccion

    - limit: define cual es el numero de imagenes con que trabajara en todo el flujo. Por defecto es -1
    - res_x: resolucion al eje x, por defecto 512
    - res_y: resolucion al eje y, por defecto 512

NOTA: Esta funcion generara un reporte de demora en cada uno de las funciones de ejecucion,
como tambien de la demora general en el proceso

'''
def pipeline(limit = -1, res_x = 512, res_y = 512):
    start = time.time()

    elapsed_f_preconditions = None
    elapsed_f_get_image_list_from_path = None
    elapsed_f_save_resized_images_resx_resy = None
    elapsed_f_sobel_filter = None
    elapsed_f_detect_vehicle_license_plate = None

    print("-----------------------")
    print("Pre-condiciones")
    try:
        start_preconditions = time.time()
        delete_dir('out_sobel')
        delete_dir('out{}x{}'.format(res_x, res_y))
        delete_dir('out_detection')
        end_preconditions = time.time()
        elapsed_f_preconditions = end_preconditions - start_preconditions
        stopwatch(elapsed_f_preconditions, 'pre-conditions block')
    except:
      print("An exception was throw executing pre-conditions block")
    print("-----------------------")



    print("Obtencion de imagenes")
    img_from_path = None
    try:
        start_f_get_image_list_from_path = time.time()
        img_from_path = get_image_list_from_path("patentes", [".png", ".jpeg"], limit)
        end_f_get_image_list_from_path = time.time()
        elapsed_f_get_image_list_from_path = end_f_get_image_list_from_path - start_f_get_image_list_from_path
        stopwatch(elapsed_f_get_image_list_from_path, 'get_image_list_from_path')
    except:
      print("An exception was throw executing 'get_image_list_from_path' function")
    print("-----------------------")

    print("Ajuste de resolucion")
    resized_images = None
    try:
        start_f_save_resized_images_resx_resy = time.time()
        resized_images = save_resized_images_resx_resy(res_x, res_y, img_from_path)
        end_f_save_resized_images_resx_resy = time.time()
        elapsed_f_save_resized_images_resx_resy = end_f_save_resized_images_resx_resy - start_f_save_resized_images_resx_resy
        stopwatch(elapsed_f_save_resized_images_resx_resy, 'save_resized_images_resx_resy')
    except:
      print("An exception was throw executing 'save_resized_images_resx_resy' function")
    print("-----------------------")

    print("Filtro Sobel")
    sobel_filter_images = None
    try:
        start_f_sobel_filter = time.time()
        sobel_filter_images = sobel_filter(resized_images)
        end_f_sobel_filter = time.time()
        elapsed_f_sobel_filter = end_f_sobel_filter - start_f_sobel_filter
        stopwatch(elapsed_f_sobel_filter, 'sobel_filter')
    except:
      print("An exception was throw executing 'sobel_filter' function")
    print("-----------------------")

    print("Deteccion figuras")
    try:
        start_f_detect_vehicle_license_plate = time.time()
        detect_vehicle_license_plate(sobel_filter_images)
        end_f_detect_vehicle_license_plate = time.time()
        elapsed_f_detect_vehicle_license_plate = end_f_detect_vehicle_license_plate - start_f_detect_vehicle_license_plate
        stopwatch(elapsed_f_detect_vehicle_license_plate, 'detect_vehicle_license_plate')
    except:
      print("An exception was throw executing 'detect_vehicle_license_plate' function")
    print("-----------------------")
    print("")

    elapsed = [elapsed_f_preconditions,
               elapsed_f_get_image_list_from_path,
               elapsed_f_save_resized_images_resx_resy,
               elapsed_f_sobel_filter,
               elapsed_f_detect_vehicle_license_plate]

    print("Metrics execution")
    print("Max time execution: {0}".format(np.amax(elapsed)))
    print("Min time execution: {0}".format(np.amin(elapsed)))
    print("Avg time execution: {0}".format(np.average(elapsed)))
    print("Std-Dev(+-) execution: {0}".format(np.std(elapsed)))
    print("")

    end = time.time()
    stopwatch(end - start, 'pipeline')

In [375]:
pipeline()

-----------------------
Pre-condiciones
Time Lapsed = 0:0:0.0009441375732421875 in function 'pre-conditions block'
-----------------------
Obtencion de imagenes
Time Lapsed = 0:0:0.004998922348022461 in function 'get_image_list_from_path'
-----------------------
Ajuste de resolucion
Time Lapsed = 0:0:1.1452720165252686 in function 'save_resized_images_resx_resy'
-----------------------
Filtro Sobel
Time Lapsed = 0:0:1.7903649806976318 in function 'sobel_filter'
-----------------------
Deteccion figuras
Time Lapsed = 0:0:0.7739520072937012 in function 'detect_vehicle_license_plate'
-----------------------

Metrics execution
Max time execution: 1.7903649806976318
Min time execution: 0.0009441375732421875
Avg time execution: 0.7431064128875733
Std-Dev(+-) execution: 0.6863011936147936

Time Lapsed = 0:0:3.719256639480591 in function 'pipeline'
